In [7]:
import collections
import gensim
import numpy as np
import pandas as pd
from tqdm import tqdm
import treetaggerwrapper
import os
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')


def wakatiWithTT(txt):
    tagdir = os.getenv('TREETAGGER_ROOT')
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='en', TAGDIR=tagdir)
    tags = tagger.TagText(txt)
    extractedTag = [tag.split()[2]  for tag in tags]
    return extractedTag


# train Data
data = pd.read_csv('./data/train.csv')
attrs = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# wakati = [wakatiWithTT(d) for d in tqdm(data['comment_text'][0:500])]
# trainData = data[0:500]
# trainData['testwakati'] = wakati
# trainData.to_csv('./data/experiment.csv')

100%|████████████████████████████████████████| 500/500 [06:56<00:00,  1.20it/s]
C:\Users\033041004\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
import pandas as pd


data = pd.read_csv('./data/experiment.csv')
attrs = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
tags = list()

for i in range(data['toxic'].size):
    tag = ''
    for a in attrs:
        if(data.iloc[i][a] == 1):
            tag += a + '-AND-'
    if(tag == ''):
        tag = 'normal'
    tags.append(tag)
data['tag'] = pd.Series(tags)
data = data.drop('comment_text', axis=1)
data.to_csv('./data/experiment2.csv')


In [ ]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd


data = pd.read_csv('./data/experiment2.csv')

# traindataの準備
trainData = list()
for i in range(data['toxic'].size):
    td = TaggedDocument(words=data.iloc[i]['testwakati'], tags=data.iloc[i]['tag'])
    trainData.append(td)

# learn
model = Doc2Vec(documents=trainData, min_count=1, dm=0)
model.save('./data/d2vSampleModel.model')
print('finished')